In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.8.0


In [4]:
print(help(tf.lite.TFLiteConverter))

Help on class TFLiteConverterV2 in module tensorflow.lite.python.lite:

class TFLiteConverterV2(TFLiteFrozenGraphConverterV2)
 |  TFLiteConverterV2(funcs, trackable_obj=None)
 |  
 |  Converts a TensorFlow model into TensorFlow Lite model.
 |  
 |  Attributes:
 |    optimizations: Experimental flag, subject to change. Set of optimizations to
 |      apply. e.g {tf.lite.Optimize.DEFAULT}. (default None, must be None or a
 |      set of values of type `tf.lite.Optimize`)
 |    representative_dataset: A generator function used for integer quantization
 |      where each generated sample has the same order, type and shape as the
 |      inputs to the model. Usually, this is a small subset of a few hundred
 |      samples randomly chosen, in no particular order, from the training or
 |      evaluation dataset. This is an optional attribute, but required for full
 |      integer quantization, i.e, if `tf.int8` is the only supported type in
 |      `target_spec.supported_types`. Refer to `tf.

### Python API 
Helper code: To identify the installed TensorFlow version, run print(tf.__version__) and to learn more about the TensorFlow Lite converter API, run print(help(tf.lite.TFLiteConverter)).

If you've installed TensorFlow 2.x, you have the following two options: (if you've installed TensorFlow 1.x, refer to Github)

Convert a TensorFlow 2.x model using tf.lite.TFLiteConverter. A TensorFlow 2.x model is stored using the SavedModel format and is generated either using the high-level tf.keras.* APIs (a Keras model) or the low-level tf.* APIs (from which you generate concrete functions). As a result, you have the following three options (examples are in the next few sections):

  tf.lite.TFLiteConverter.from_saved_model() (recommended): Converts a SavedModel.
  - `tf.lite.TFLiteConverter.from_keras_model()`: Converts a Keras model.
  - `tf.lite.TFLiteConverter.from_concrete_functions()`: Converts concrete functions.

Convert a TensorFlow 1.x model using tf.compat.v1.lite.TFLiteConverter (examples are on Github):

  - `tf.compat.v1.lite.TFLiteConverter.from_saved_model()`: Converts a SavedModel.
  - `tf.compat.v1.lite.TFLiteConverter.from_keras_model_file()`: Converts a Keras model.
  - `tf.compat.v1.lite.TFLiteConverter.from_session()`: Converts a GraphDef from a session.
  - `tf.compat.v1.lite.TFLiteConverter.from_frozen_graph()`: Converts a Frozen GraphDef from a file. If you have checkpoints, then first convert it to a Frozen GraphDef file and then use this API as shown here.

In [11]:
# unzip your model
!unzip /content/saved_model.zip -d /content/saved_model

Archive:  /content/saved_model.zip
   creating: /content/saved_model/content/saved_model/1647401327/
   creating: /content/saved_model/content/saved_model/1647401327/assets/
  inflating: /content/saved_model/content/saved_model/1647401327/saved_model.pb  
  inflating: /content/saved_model/content/saved_model/1647401327/keras_metadata.pb  
   creating: /content/saved_model/content/saved_model/1647401327/variables/
  inflating: /content/saved_model/content/saved_model/1647401327/variables/variables.index  
  inflating: /content/saved_model/content/saved_model/1647401327/variables/variables.data-00000-of-00001  


### Convert a SavedModel (recommended)

In [14]:
# Convert the model 
saved_model_dir = '/content/saved_model/content/saved_model/1647401327'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the model
with open('model_saved.tflite','wb') as f:
  f.write(tflite_model)

### Convert a Keras model

In [16]:
# Create a model using high-level tf.keras.*APIs
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(units = 1, input_shape = [1]),
                                    tf.keras.layers.Dense(units = 16,activation = 'relu'),
                                    tf.keras.layers.Dense(units = 1)
])

# compile the model
model.compile(loss = 'mean_squared_error',
              optimizer = 'sgd',
              )
# train your model
model.fit(x = [-1,0,1],y = [-3,-1,1],epochs = 5) # train the model
# to generate SavedModel tf.saved_model.save(model,'saved_model_keras_dir')

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save your model
with open('model_keras.tflite','wb') as f:
  f.write(tflite_model)

Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 3.6388
Epoch 2/5
1/1 [==============================] - 0s 10ms/step - loss: 3.5429
Epoch 3/5
1/1 [==============================] - 0s 11ms/step - loss: 3.4408
Epoch 4/5
1/1 [==============================] - 0s 11ms/step - loss: 3.3428
Epoch 5/5
1/1 [==============================] - 0s 15ms/step - loss: 3.2487
INFO:tensorflow:Assets written to: /tmp/tmp28ac4x4u/assets


INFO:tensorflow:Assets written to: /tmp/tmp28ac4x4u/assets


### Convert concrete functions

In [22]:
# Create a model using low-level tf.*APIs
class Squared(tf.Module):
  @tf.function(input_signature = [tf.TensorSpec(shape= [None],dtype = tf.float32)])
  def __call__(self,x):
    return tf.square(x) 

model = Squared()

# (to generate a SavedModel) tf.saved_model_save(model,'saved_model_tf_dir")
concrete_func = model.__call__.get_concrete_function()
concrete_func

<ConcreteFunction __call__(x) at 0x7F145C1DEB90>

In [24]:
# Convert the model
# Notes that for the versions earlier than TensorFlow 2.7, the
# from_concrete_functions API is able to work when there is only the first
# argument given:
# > converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func],
                                                            model
                                                            )

tflite_model = converter.convert()

# Saved the model
with open('model_concrete.tflite','wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp1p_ze_2y/assets


INFO:tensorflow:Assets written to: /tmp/tmp1p_ze_2y/assets


### Command line Tool

In [26]:
!tflite_convert --help

2022-03-16 04:00:30.854514: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
usage: tflite_convert [-h] --output_file OUTPUT_FILE
                      [--saved_model_dir SAVED_MODEL_DIR | --keras_model_file KERAS_MODEL_FILE]
                      [--saved_model_tag_set SAVED_MODEL_TAG_SET]
                      [--saved_model_signature_key SAVED_MODEL_SIGNATURE_KEY]
                      [--enable_v1_converter]
                      [--experimental_new_converter [EXPERIMENTAL_NEW_CONVERTER]]
                      [--experimental_new_quantizer [EXPERIMENTAL_NEW_QUANTIZER]]

Command line tool to run TensorFlow Lite Converter.

optional arguments:
  -h, --help            show this help message and exit
  --output_file OUTPUT_FILE
                        Full filepath of the output file.
  --saved_model_dir SAVED_MODEL_DIR
                        Full path of the directory containing the SavedModel.
  --k

In [32]:
# convert SavedModel
!tflite_convert \
--saved_model_dir= /content/saved_model/content/saved_model/1647401327 \
--output_file =/content/model_saved2.tflite

2022-03-16 04:07:25.037898: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
usage: tflite_convert [-h] --output_file OUTPUT_FILE
                      [--saved_model_dir SAVED_MODEL_DIR | --keras_model_file KERAS_MODEL_FILE]
                      [--saved_model_tag_set SAVED_MODEL_TAG_SET]
                      [--saved_model_signature_key SAVED_MODEL_SIGNATURE_KEY]
                      [--enable_v1_converter]
                      [--experimental_new_converter [EXPERIMENTAL_NEW_CONVERTER]]
                      [--experimental_new_quantizer [EXPERIMENTAL_NEW_QUANTIZER]]
tflite_convert: error: one of the arguments --saved_model_dir --keras_model_file is required


In [ ]:
# Convert
tflite_convert \
  --keras_model_file=/tmp/mobilenet_keras_model.h5 \
  --output_file=/tmp/mobilenet.tflite